Test bodo snowflake connectivity

In [3]:

import snowflake.connector
#import bodo
import time
from sqlalchemy.dialects import registry
registry.register('snowflake', 'snowflake.sqlalchemy', 'dialect')

In [4]:

import os
with open("./snowflake-secrets.txt","r") as f:
    for line in f:
        args=line.split("=")
        os.environ[args[0]]=args[1].strip()
username=os.environ["username"]
password=os.environ["password"]
account=os.environ["account"]
warehouse="BODO_VW"
database="BODO_DB"
role="BODO_R1"

In [6]:

import bodo
import pandas as pd
@bodo.jit()
def getdata():
    print("running")
    df=pd.read_sql("""select L_ORDERKEY , L_PARTKEY , L_SUPPKEY , L_LINENUMBER , L_QUANTITY , L_EXTENDEDPRICE , L_DISCOUNT , L_TAX , L_RETURNFLAG,
    L_LINESTATUS, L_SHIPDATE DATE, L_COMMITDATE, TEST_DATE, L_SHIPINSTRUCT , L_SHIPMODE , L_COMMENT , DUMMY, TEST_TS, TEST_DOUBLE , TEST_FLOAT ,
    TEST_BOOLEAN, TEST_NUMBER, TEST_VARIANT, TEST_OBJECT , TEST_ARRAY, TEST_INTEGER , TEST_TINYINT , TEST_BYTEINT , TEST_DECIMAL, TEST_NUMERIC, TEST_CHAR, TEST_CHARACTER,
    TEST_STRING, TEST_TEXT, TEST_BIGINT , TEST_SMALLINT , TEST_FLOAT4 , TEST_FLOAT8 , TEST_DOUBLE_PRECISION , TEST_REAL , TEST_DATETIME, TEST_TIMESTAMP_NTZ, TEST_BINARY,
    TEST_VARCHAR1, TEST_VARCHAR2, TEST_VARCHAR3, TEST_GEOGRAPHY  from BODO_DB.public.TEST_DATATYPEVALIDATION limit 10""", f"snowflake://{username}:{password}@{account}/{database}/public?role={role}&warehouse={warehouse}",)
   
    return df
df=getdata()
df["test_date"]=df["test_date"].astype(str)
df["test_varchar1"]=pd.to_datetime(df["test_varchar1"],format="%Y-%M-%d")
df["test_float"]=df["test_float"].astype(str)
df["test_varchar2"]=pd.to_numeric(df["test_varchar2"])
df["test_integer"]=df["test_integer"].astype(str)
df["test_varchar3"]=pd.to_numeric(df["test_varchar3"])
print(df.dtypes)
df[["test_date","test_float","test_integer","test_varchar1","test_varchar2","test_varchar3"]]


running
l_orderkey                        Int32
l_partkey                         Int16
l_suppkey                          Int8
l_linenumber                       Int8
l_quantity                      float64
l_extendedprice                 float64
l_discount                      float64
l_tax                           float64
l_returnflag                     object
l_linestatus                     object
date                             object
l_commitdate                     object
test_date                        object
l_shipinstruct                   object
l_shipmode                       object
l_comment                        object
dummy                            object
test_ts                  datetime64[ns]
test_double                     float64
test_float                       object
test_boolean                    boolean
test_number                     float64
test_variant                     object
test_object                      object
test_array                      

,test_date,test_float,test_integer,test_varchar1,test_varchar2,test_varchar3
0,1996-03-22,100.1,1,1996-01-22 00:03:00,100.1,1
1,1996-04-20,100.1,1,1996-01-20 00:04:00,100.1,1
2,1996-01-31,100.1,1,1996-01-31 00:01:00,100.1,1
3,1996-05-16,100.1,1,1996-01-16 00:05:00,100.1,1
4,1996-04-01,100.1,1,1996-01-01 00:04:00,100.1,1
5,1996-02-03,100.1,1,1996-01-03 00:02:00,100.1,1
6,1997-02-02,100.1,1,1997-01-02 00:02:00,100.1,1
7,1994-02-23,100.1,1,1994-01-23 00:02:00,100.1,1
8,1993-11-24,100.1,1,1993-01-24 00:11:00,100.1,1
9,1994-01-23,100.1,1,1994-01-23 00:01:00,100.1,1


In [ ]:
def parse_variant(df):
    df=pd.DataFrame(df)
    return df
df3=df["test_variant"].apply(parse_variant)
df3

In [15]:
import json

def parse_json(x):
    obj=json.loads(x)
    return pd.Series(obj)

import json
def parse_scores(x):
    x["score"]=x["scores"]["score"]
    x["type"]=x["scores"]["type"]
    return x
df[["name","scores","u_id"]]=df["test_variant"].apply(parse_json)
df

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,...,test_datetime,test_timestamp_ntz,test_binary,test_varchar1,test_varchar2,test_varchar3,test_geography,name,scores,u_id
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 57.24794864351232, 'type': 'exam'},...",76
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 57.24794864351232, 'type': 'exam'},...",76
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 57.24794864351232, 'type': 'exam'},...",76
3,1,22,48,4,28.0,25816.56,0.09,0.06,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-16 00:05:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 57.24794864351232, 'type': 'exam'},...",76
4,1,241,23,5,24.0,27389.76,0.10,0.04,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-01 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 57.24794864351232, 'type': 'exam'},...",76
5,1,157,10,6,32.0,33828.80,0.07,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-03 00:02:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 57.24794864351232, 'type': 'exam'},...",76
6,2,1062,33,1,38.0,36596.28,0.00,0.05,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1997-01-02 00:02:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"[{'score': 64.16109192679477, 'type': 'exam'},...",190
7,3,43,19,1,45.0,42436.80,0.06,0.00,R,F,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1994-01-23 00:02:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Aleida Elsass,"[{'score': 28.02518041693717, 'type': 'exam'},...",41
8,3,191,70,2,49.0,53468.31,0.10,0.00,R,F,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1993-01-24 00:11:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Aleida Elsass,"[{'score': 28.02518041693717, 'type': 'exam'},...",41
9,3,1285,60,3,27.0,32029.56,0.06,0.07,A,F,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1994-01-23 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Aleida Elsass,"[{'score': 28.02518041693717, 'type': 'exam'},...",41


In [16]:
df=df.explode(column="scores")
df

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,...,test_datetime,test_timestamp_ntz,test_binary,test_varchar1,test_varchar2,test_varchar3,test_geography,name,scores,u_id
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 19.5118228072558, 'type': 'quiz'}",76
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 70.71043448913191, 'type': 'homework'}",76
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 19.5118228072558, 'type': 'quiz'}",76
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 70.71043448913191, 'type': 'homework'}",76
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 19.5118228072558, 'type': 'quiz'}",76
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 70.71043448913191, 'type': 'homework'}",76
3,1,22,48,4,28.0,25816.56,0.09,0.06,N,O,...,2020-05-04 04:29:11.709,2020-05-04 04:29:11.709,b'\x1d\xe9i',1996-01-16 00:05:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76


In [17]:
df=df.apply(parse_scores,axis=1)


In [18]:
df

,l_orderkey,l_partkey,l_suppkey,l_linenumber,l_quantity,l_extendedprice,l_discount,l_tax,l_returnflag,l_linestatus,...,test_binary,test_varchar1,test_varchar2,test_varchar3,test_geography,name,scores,u_id,score,type
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76,57.247949,exam
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 19.5118228072558, 'type': 'quiz'}",76,19.511823,quiz
0,1,1552,93,1,17.0,24710.35,0.04,0.02,N,O,...,b'\x1d\xe9i',1996-01-22 00:03:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 70.71043448913191, 'type': 'homework'}",76,70.710434,homework
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76,57.247949,exam
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 19.5118228072558, 'type': 'quiz'}",76,19.511823,quiz
1,1,674,75,2,36.0,56688.12,0.09,0.06,N,O,...,b'\x1d\xe9i',1996-01-20 00:04:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 70.71043448913191, 'type': 'homework'}",76,70.710434,homework
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76,57.247949,exam
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 19.5118228072558, 'type': 'quiz'}",76,19.511823,quiz
2,1,637,38,3,8.0,12301.04,0.10,0.02,N,O,...,b'\x1d\xe9i',1996-01-31 00:01:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 70.71043448913191, 'type': 'homework'}",76,70.710434,homework
3,1,22,48,4,28.0,25816.56,0.09,0.06,N,O,...,b'\x1d\xe9i',1996-01-16 00:05:00,100.1,1,"{\n ""coordinates"": [\n -122.35,\n 37.55...",Adrien Renda,"{'score': 57.24794864351232, 'type': 'exam'}",76,57.247949,exam
